In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [32]:
train = pd.read_csv('/content/drive/MyDrive/spell-checker/Копия train.csv', usecols=['corrupted_text', 'correct_text'])

Удаляем пунктуацию, переводим текст в нижний регистр и составляем список слов из данной строки

In [36]:
train.sample(5)

,corrupted_text,correct_text
612568,"Слушай, я пыатюст найти ее, помовории с ней.","Слушай, я пытаюсь найти ее, поговорить с ней."
139020,"Лотии, ты дкржи пацана окренче.","Лотии, ты держи пацана покрепче."
635107,сфтоед Борщ черниговский,Сытоедов Борщ черниговский
1133162,Я понимаю это. Этр всего-лишь ноктбк.,Я понимаю это. Это всего-лишь ноутбук.
1359494,Нет. Почему вы беоете нш телевизор?,Нет. Почему вы берете наш телевизор?


In [5]:
def remove_punctuation(text):
    punctuation = '''!@#$%^&*(){}[]|._`/?:;"'\,~'''
    for symbol in punctuation:
        if symbol in text:
            text = text.replace(symbol, '')
    return text.strip()

In [6]:
train['corrupt_text_no_punctuation'] = train.corrupted_text.apply(lambda x: remove_punctuation(x)).apply(lambda x: x.lower())
train['corrupt_text_list'] = train['corrupt_text_no_punctuation'].apply(lambda x: x.split())

In [7]:
train['correct_text_no_punctuation'] = train.correct_text.apply(lambda x: remove_punctuation(x)).apply(lambda x: x.lower())
train['correct_text_list'] = train['correct_text_no_punctuation'].apply(lambda x: x.split())

Создаем словари для слов в которых были совершены ошибки, для корректных слов и для биграм (словосочетаний по 2 слова)

In [8]:
bad_words = {} #слова с ошибками
good_words = {} #валидные слова
bigrams = {} #биграммы

In [9]:
# функция для составления словаря биграмм
def make_bigrams(text):
    for i in range(len(text) - 1):
        bigram = text[i] + ' ' + text[i + 1]
        if bigram in bigrams:
            bigrams[bigram] += 1
        else:
            bigrams[bigram] = 1

'''функция для составления словаря валидных слов, где ключ - слово,
значение - сколько раз оно встретилось'''
def make_good_dict(text):
    for word in text:
        if word in good_words:
            good_words[word] += 1
        else:
            good_words[word] = 1

'''функция для составления словаря слов с ошибками (сопоставляет слову
из текста с ошибками слово на той же позиции из корректного текста)'''
def make_bad_dict(correct, corrupt):
    for good_word, bad_word in zip(correct, corrupt):
        if good_word != bad_word and good_word in good_words:
            if bad_word in bad_words:
                bad_words[bad_word].add(good_word)
            else:
                bad_words[bad_word] = {good_word}

In [10]:
train.correct_text_list.apply(lambda x: make_good_dict(x))
train.apply(lambda x: make_bad_dict(x.correct_text_list, x.corrupt_text_list), axis=1)
train.correct_text_list.apply(lambda x: make_bigrams(x))

0          None
1          None
2          None
3          None
4          None
           ... 
1480415    None
1480416    None
1480417    None
1480418    None
1480419    None
Name: correct_text_list, Length: 1480420, dtype: object

Используем вероятностный подход для оценки контекста и вероятности употребления данного слова

In [11]:
'''Сглаженная функция (не переходит в ноль)
   для определения вероятности появления слов'''
def pdist_additive_smoothed(counter, c=1):  #counter - словарь токенов и их частот
    N = sum(list(counter.values()))          # суммарное кол-во слов
    Nplus = N + c * (len(counter) + 1) # кол-во слов + сглаживание
    return lambda word: (counter[word] + c) / Nplus

P1 = pdist_additive_smoothed(good_words)
P2 = pdist_additive_smoothed(bigrams)

# функция для перемножения вероятностей
def product(nums):
    result = 1
    for x in nums:
        result *= x
    return result

In [12]:
# вероятность последовательности слов с помощью биграммной модели (при условии предыдущего слова)
def Pwords2(words, prev=''):
    return product(cPword(w, (prev if (i == 0) else words[i-1]) )
                   for (i, w) in enumerate(words))

# условная вероятность слова при условии предыдущего
def cPword(word, prev):
    "Условная вероятность слова при условии предыдущего."
    bigram = prev + ' ' + word
    if bigram in bigrams and prev in good_words:
        return P2(bigram) / P1(prev)
    else: # если что-то не встретилось, поставим среднее между P1 и 0
        return P1(word) / 2

Определяем возможные валидные слова на edit distance 0, 1 и 2 от нашего слова

In [13]:
''' функция возвращает подмножество слов, которое есть в нашем словаре с частотой более 10 '''
def known(words):
    res = set()
    for w in words:
        if w in good_words.keys() and good_words[w] > 10:
            res.add(w)
    return res

# все строки, которые находятся на edit_distance == 0 от word (т.е само слово)
def edits0(word):
    return {word}

# все строки, которые находятся на edit_distance == 2 от word
def edits2(word):
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [14]:
# возвращает список всех строк на расстоянии edit_distance == 1 от word
def edits1(word):
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

# список всех возможных разбиений слова на пару (a, b)
def splits(word):
    return [(word[:i], word[i:])
            for i in range(len(word)+1)]

# включили - в алфавит, т.к есть слова по типу "какой-то", "посмтри-ка"
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя-'

Осуществляем исправления слов.
Сначала прямолинейным подходом - ищем в словаре неправильных слов замену (если несколько вариантов исправления, выбираем самый подходящий по биграммам или вероятности появления данного слова).
Далее пользуемся подходом Норвига (вышенаписанные функции) с изменением неправильного слова на расстояния 1 и 2 (если получилось несколько кандидатов также смотрим по биграммам и вероятностям слов)

In [15]:
# поиск наиболее вероятной биграммы
def find_best_b(bi):
    best_P = 0
    for b in bi:
        if Pwords2(b) >= best_P:
            best_P = Pwords2(b)
            best_bi = b
    return best_bi

In [16]:
# реализация прямолинейного подхода
def checker(text_check):
    text = text_check.copy()
    for i in range(len(text)):
        ''' ищем слова которые были исправлены в train и не содержатся в словаре валидных слов
        (чтобы случайно не исправить правильные слова)'''
        if text[i] in bad_words and (text[i] not in good_words or good_words[text[i]] < 100):
            candidates = bad_words[text[i]]
            # если предыдущее слово - валидное, составляем с ним биграммы
            if i != 0 and text[i - 1] in good_words:
                bi = []
                for candidate in candidates:
                    bi.append([text[i - 1], candidate])

                best_bi = find_best_b(bi)
                text[i] = best_bi[1]
            # если предыдущее слово - не валидное, смотрим по вероятности текущего слова
            else:
                text[i] = (max(bad_words[text[i]], key=P1))

    return text

In [17]:
# реализация подхода с исправлением опечаток на расстоянии 1 и 2
def checker2(text_check):
    text = text_check.copy()
    for i in range(len(text)):
        # используем только для некорректных слов
        if text[i] not in good_words:
            # выбираем кандидатов с помощью написанных выше функций
            candidates = (known(edits1(text[i])) or known(edits2(text[i])))
            if not candidates:
                continue
            if i != 0 and text[i - 1] in good_words and len(text[i]) > 3:
                bi = []
                for candidate in candidates:
                    bi.append([text[i - 1], candidate])
                best_bi = find_best_b(bi)
                text[i] = best_bi[1]
            else:
                text[i] = max(candidates, key=good_words.get)

    return text

In [18]:
# восстановление пунктуации
def restore_punctuation(original_text, good_text):
    res = []
    punctuation = '''!@#$%^&*(){}[]|._`/?:;"'\,~'''
    for orig_word, word in zip(original_text.split(), good_text):
        if orig_word[-1] in punctuation:
            word = word + orig_word[-1]
        if orig_word[0] in punctuation:
            word = orig_word[0] + word

        if (orig_word == '-' or orig_word == '—') and word == 'я':
            word = orig_word

        if orig_word.istitle():
            word = word.capitalize()

        res.append(word)

    restored_text = ' '.join(res)
    for punct in punctuation:
        restored_text = restored_text.replace(' ' + punct, punct)

    return restored_text

In [37]:
texts = [
    'Это с тобой жлучилaсь, Алекс?',
    'Я не - Лучше рпсядьте.',
    'плетег шнур Aqualon',
    'Рожок гиант колибри',
    'У меня там когда-то зсрряла штанега.',
    'Слушай, я пыатюст найти ее, помовории с ней.'
]

In [38]:
from IPython.display import display, HTML

# для подкрашивания измененных слов в получившемся тексте
def highlight_corrections(original_text, corrected_text):
    original_words = original_text.split()
    corrected_words = corrected_text.split()

    def create_highlighted_html(words, corrections):
        highlighted_html = ""
        for word, is_corrected in zip(words, corrections):
            if is_corrected:
                highlighted_html += f"<mark>{word}</mark> "
            else:
                highlighted_html += word + " "
        return highlighted_html

    corrections = [ow != cw for ow, cw in zip(original_words, corrected_words)]
    highlighted_original = create_highlighted_html(original_words, corrections)
    highlighted_corrected = create_highlighted_html(corrected_words, corrections)

    return f"Оригинальный текст: {highlighted_original}<br><br>Исправленный текст: {highlighted_corrected}"

In [39]:
def inference(text):
    text_fix = remove_punctuation(text)
    text_fix = text_fix.split()
    text_fix = checker(text_fix)
    correct_text = checker2(text_fix)
    correct_text_punkt = restore_punctuation(text, correct_text)
    higlight_text = highlight_corrections(text, correct_text_punkt)
    display(HTML(higlight_text))

In [40]:
for text in texts:
    inference(text)
    print('-' * 100)

----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------
